In [7]:
# Importacion de librerias
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Configura una sesión de caché para las solicitudes HTTP con una duración de caché infinita (-1)
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)

# Configura una sesión de solicitudes con reintentos automáticos en caso de fallos, hasta 5 reintentos con un factor de retroceso de 0.2
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)

# Crea un cliente de openmeteo_requests utilizando la sesión con caché y reintentos
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define la URL del API de Open Meteo y los parámetros para la solicitud
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 40.7143,  # Latitud de la ubicación (Nueva York)
    "longitude": -74.006,  # Longitud de la ubicación (Nueva York)
    "start_date": "2017-01-01",  # Fecha de inicio para los datos históricos
    "end_date": "2023-12-31",  # Fecha de fin para los datos históricos
    "daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max"],  # Variables diarias a solicitar
    "timezone": "America/New_York"  # Zona horaria
}

# Realiza la solicitud al API con los parámetros especificados
responses = openmeteo.weather_api(url, params=params)

# Obtiene la primera respuesta del conjunto de respuestas
response = responses[0]

# Obtiene los datos diarios de la respuesta
daily = response.Daily()

# Extrae las variables diarias como arrays de Numpy
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(5).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(6).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(7).ValuesAsNumpy()

# Crea un diccionario para almacenar los datos diarios con sus fechas correspondientes
daily_data = {"date": pd.date_range(
    start=pd.to_datetime(daily.Time(), unit="s", utc=True).date(),
    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True).date(),
    freq=pd.Timedelta(seconds=daily.Interval()),
    inclusive="left"
)}

# Añade los datos de las variables diarias al diccionario
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max

# Convierte el diccionario a un DataFrame de pandas
daily_dataframe = pd.DataFrame(data=daily_data)

#Guarda el dataframe en un archivo csv
csv_file_path = "../Data/daily_weather_data.csv"
daily_dataframe.to_csv(csv_file_path, index=False)